# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import math
import xlsxwriter
import requests

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [29]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [17]:
from api_token import IEX_CLOUD_API_TOKEN
from consts import TRILLION


pk_4aab44f8cbd5436aa3320df8ac01dbf3


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [18]:
api_url = "https://api.iex.cloud/v1"


def get_curr_stock_price(ticker : str):
    endpoint = f'{api_url}/data/core/quote/{ticker}?token={IEX_CLOUD_API_TOKEN}'
    r = requests.get(url=endpoint)
    if (r.status_code != 200):
        return -1
    r = r.json()[0].get("iexClose")
    return (r if r else -1)

def get_curr_stock_market_cap(ticker : str) -> float:
    endpoint = f'{api_url}/data/core/quote/{ticker}?token={IEX_CLOUD_API_TOKEN}'
    r = requests.get(url=endpoint)
    if (r.status_code != 200):
        return -1
    r = r.json()[0].get("marketCap")
    return (r if r else -1)

        
print(f'current stock price: {get_curr_stock_price("aapl")}')
print(f'current stock market cap (in Trillions): {get_curr_stock_market_cap("aapl")/TRILLION}')



current stock price: 186.335
current stock market cap (in Trillions): 2.89802204392


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [36]:
def create_rows(tickers : list[str]):
    # use batch process
    endpoint = f'{api_url}/data/core/quote/{",".join(tickers)}?token={IEX_CLOUD_API_TOKEN}'
    r = requests.get(url=endpoint)
    if (r.status_code != 200):
        return -1
    quote_list = r.json()
    return [[q.get("companyName"), q.get("symbol"), q.get("iexClose"), q.get("marketCap"), "N/A"] for q in quote_list]

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [43]:
df_columns_name = ["Company Name", "Ticker", "Price", "MarketCap", "Number of Shares"]
ticker_list = stocks.get("Ticker").tolist()
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [49]:
sp_df = pd.DataFrame(columns = df_columns_name)

for tl in chunks(ticker_list, 100):
    sp_df = pd.concat([sp_df, create_rows(tl)])

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [48]:
display(sp_df)

,Company Name,Ticker,Price,MarketCap,Number of Shares,0,1,2,3,4,...,90,91,92,93,94,95,96,97,98,99
0,NaN,NaN,NaN,NaN,NaN,"[Agilent Technologies Inc., A, 108.44, 3173694...","[American Airlines Group Inc, AAL, 11.79, 7705...","[Advance Auto Parts Inc, AAP, 56.13, 333806575...","[Apple Inc, AAPL, 186.335, 2898022043920, N/A]","[Abbvie Inc, ABBV, 138.61, 244685831176, N/A]",...,"[Citizens Financial Group Inc, CFG, 25.175, 11...","[Church & Dwight Co., Inc., CHD, 89.72, 221053...","[C.H. Robinson Worldwide, Inc., CHRW, 79.93, 9...","[Charter Communications Inc. - Class A, CHTR, ...","[Cigna Group (The), CI, 293.31, 85799100231, N/A]","[Cincinnati Financial Corp., CINF, 99.73, 1564...","[Colgate-Palmolive Co., CL, 75.42, 62106969496...","[Clorox Co., CLX, 133.52, 16556927219, N/A]","[Comerica, Inc., CMA, 41.36, 5452940776, N/A]","[Comcast Corp - Class A, CMCSA, 41.89, 1682149..."
1,NaN,NaN,NaN,NaN,NaN,"[CME Group Inc - Class A, CME, 217.14, 7816819...","[Chipotle Mexican Grill, CMG, 2119.82, 5819859...","[Cummins Inc., CMI, 219.07, 31044959894, N/A]","[CMS Energy Corporation, CMS, 55.62, 162220543...","[Centene Corp., CNC, 71.87, 38393025870, N/A]",...,"[Fox Corporation - Class B, FOX, 28.09, 140584...","[Fox Corporation - Class A, FOXA, 30.12, 14068...","[First Republic Bank, FRC, 3.5, 630560307, N/A]","[Federal Realty Investment Trust., FRT, 92.545...","[TechnipFMC plc, FTI, 21.58, 9384166620, N/A]","[Fortinet Inc, FTNT, 50.83, 39032845120, N/A]","[Fortive Corp, FTV, 65.82, 23134871847, N/A]","[General Dynamics Corp., GD, 243.55, 664640302...","[General Electric Co., GE, 115.27, 12545825917...","[Gilead Sciences, Inc., GILD, 75.9, 9457457983..."
2,NaN,NaN,NaN,NaN,NaN,"[General Mills, Inc., GIS, 65.3, 37945908069, ...","[Globe Life Inc, GL, 116.64, 10978009834, N/A]","[Corning, Inc., GLW, 27.7, 23615880651, N/A]","[General Motors Company, GM, 26.85, 3677057038...","[Alphabet Inc - Class C, GOOG, 134.11, 1552703...",...,"[Las Vegas Sands Corp, LVS, 47.76, 36504439234...","[Lamb Weston Holdings Inc, LW, 94.03, 13633287...","[LyondellBasell Industries NV - Class A, LYB, ...","[Live Nation Entertainment Inc, LYV, 87.52, 20...","[Mastercard Incorporated - Class A, MA, 394.55...","[Mid-America Apartment Communities, Inc., MAA,...","[Marriott International, Inc. - Class A, MAR, ...","[Masco Corp., MAS, 56.16, 12610216171, N/A]","[McDonald`s Corp, MCD, 267.78, 194239342214, N/A]","[Microchip Technology, Inc., MCHP, 77.6, 41985..."
3,NaN,NaN,NaN,NaN,NaN,"[Mckesson Corporation, MCK, 469.055, 633083379...","[Moody`s Corp., MCO, 344.82, 63056310000, N/A]","[Mondelez International Inc. - Class A, MDLZ, ...","[Medtronic Plc, MDT, 70.675, 94042122835, N/A]","[Metlife Inc, MET, 60.47, 44774106952, N/A]",...,"[Royal Caribbean Group, RCL, 94.91, 2431927372...","[Everest Re Group Ltd, RE, 351.36, 13796600687...","[Regency Centers Corporation, REG, 60.48, 1116...","[Regeneron Pharmaceuticals, Inc., REGN, 798.49...","[Regions Financial Corp., RF, 15.13, 140625818...","[Robert Half Inc, RHI, 76.97, 8152852585, N/A]","[Raymond James Financial, Inc., RJF, 100.49, 2...","[Ralph Lauren Corp - Class A, RL, 115.69, 7836...","[Resmed Inc., RMD, 143.67, 21122398276, N/A]","[Rockwell Automation Inc, ROK, 258.25, 2963482..."
4,NaN,NaN,NaN,NaN,NaN,"[Rollins, Inc., ROL, 38.41, 18587053747, N/A]","[Roper Technologies Inc, ROP, 516.94, 55163132...","[Ross Stores, Inc., ROST, 123.36, 42023304052,...","[Republic Services, Inc., RSG, 157.76, 4964649...","[RTX Corp, RTX, 82.21, 118224243570, N/A]",...,"[West Pharmaceutical Services, Inc., WST, 341....","[Western Union Company, WU, 11.745, 4281218344...","[Weyerhaeuser Co., WY, 30.15, 22009530150, N/A]","[Wynn Resorts Ltd., WYNN, 85.51, 9742695920, N/A]","[Xcel Energy, Inc., XEL, 59.2, 32667526085, N/A]","[Xilinx, Inc., XLNX, 194.86, 48414620999, N/A]","[Exxon Mobil Corp., XOM, 103.78, 411152730673,...","[DENTSPLY Sirona Inc, XRAY, 28.4, 6016835104, ...","[Xerox Holdings Corp, XRX, 13.36, 1642024147, ...","[Xylem Inc, XYL

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: